# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from functools import reduce
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import the CSV
cities_pd = pd.read_csv("../output_data/cities.csv")
cities_pd.head()

,Unnamed: 0,City,date,lat,lon,max_temp,clouds,wind,humidity,Country
0,0,zabol,1595986588,32.25,67.25,73.33,0,4.09,20,AF
1,1,east london,1595986588,-33.02,27.91,61.11,0,11.23,70,ZA
2,2,liberty,1595986373,31.83,-81.50,84.20,75,13.87,62,US
3,3,mangai,1595986588,-4.05,19.53,72.10,0,1.03,53,CD
4,4,gardenstown,1595986588,57.67,-2.34,54.00,100,37.00,79,GB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store 'lat' and 'lon' into  locations 
locations = cities_pd[["lat", "lon"]]

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
cities_pd = cities_pd.dropna()
humidity_rate = cities_pd["humidity"]

In [7]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#A max temperature between 70-80 degrees. 
columns = ['City', 'lat', 'lon', 'max_temp', 'Country']
temp_range = cities_pd.loc[cities_pd["max_temp"].between(70, 80, inclusive=True),  columns]
temp_range.dropna()
temp_range.head()

,City,lat,lon,max_temp,Country
0,zabol,32.25,67.25,73.33,AF
3,mangai,-4.05,19.53,72.10,CD
8,lagoa,39.05,-27.98,73.13,PT
11,gat,31.61,34.76,79.00,IL
18,albany,42.60,-73.97,75.00,US


In [9]:
#Wind speed less than 10 mph.
columns = ['City', 'lat', 'lon', 'wind', 'Country']
ideal_wind = cities_pd.loc[cities_pd["wind"] < 10,  columns]

ideal_wind.dropna()
ideal_wind.head()

,City,lat,lon,wind,Country
0,zabol,32.25,67.25,4.09,AF
3,mangai,-4.05,19.53,1.03,CD
5,akureyri,65.68,-18.09,2.24,IS
9,qaanaaq,77.48,-69.36,3.29,GL
11,gat,31.61,34.76,1.01,IL


In [10]:
#Zero cloudiness.
columns = ['City', 'lat', 'lon', 'clouds', 'Country']
zero_clouds = cities_pd.loc[cities_pd["clouds"] == 0,  columns]

zero_clouds.dropna()
zero_clouds.head()

,City,lat,lon,clouds,Country
0,zabol,32.25,67.25,0,AF
1,east london,-33.02,27.91,0,ZA
3,mangai,-4.05,19.53,0,CD
6,carnarvon,-24.87,113.63,0,AU
11,gat,31.61,34.76,0,IL


In [11]:
dfs = [temp_range, ideal_wind, zero_clouds]
reduced_df = reduce(lambda left,right: pd.merge(left,right,on='City'), dfs)
reduced_df.head()

,City,lat_x,lon_x,max_temp,Country_x,lat_y,lon_y,wind,Country_y,lat,lon,clouds,Country
0,zabol,32.25,67.25,73.33,AF,32.25,67.25,4.09,AF,32.25,67.25,0,AF
1,mangai,-4.05,19.53,72.10,CD,-4.05,19.53,1.03,CD,-4.05,19.53,0,CD
2,gat,31.61,34.76,79.00,IL,31.61,34.76,1.01,IL,31.61,34.76,0,IL
3,sao felix do xingu,-6.64,-51.99,70.43,BR,-6.64,-51.99,1.54,BR,-6.64,-51.99,0,BR
4,waddan,29.16,16.14,70.20,LY,29.16,16.14,2.93,LY,29.16,16.14,0,LY


In [12]:
reduced_df.columns

Index(['City', 'lat_x', 'lon_x', 'max_temp', 'Country_x', 'lat_y', 'lon_y',
       'wind', 'Country_y', 'lat', 'lon', 'clouds', 'Country'],
      dtype='object')

In [13]:
hotel_df = reduced_df.loc[:, ['City', 'max_temp', 'wind', 
                              'clouds', 'lat', 'lon', 'Country']]
hotel_df.head()

,City,max_temp,wind,clouds,lat,lon,Country
0,zabol,73.33,4.09,0,32.25,67.25,AF
1,mangai,72.10,1.03,0,-4.05,19.53,CD
2,gat,79.00,1.01,0,31.61,34.76,IL
3,sao felix do xingu,70.43,1.54,0,-6.64,-51.99,BR
4,waddan,70.20,2.93,0,29.16,16.14,LY


In [14]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,max_temp,wind,clouds,lat,lon,Country,Hotel Name
0,zabol,73.33,4.09,0,32.25,67.25,AF,
1,mangai,72.10,1.03,0,-4.05,19.53,CD,
2,gat,79.00,1.01,0,31.61,34.76,IL,
3,sao felix do xingu,70.43,1.54,0,-6.64,-51.99,BR,
4,waddan,70.20,2.93,0,29.16,16.14,LY,


In [15]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [18]:
hotel_df.head(10)

,City,max_temp,wind,clouds,lat,lon,Country,Hotel Name
0,lasa,70.00,2.59,0,46.62,10.70,IT,Pension Feldgärtenhof
1,lagoa,72.57,6.42,0,39.05,-27.98,PT,Graciosa Resort - Biosphere Island Hotel
2,urucara,72.86,0.54,0,-2.54,-57.76,BR,Hotel Eldorado
3,asau,79.00,0.27,0,46.43,26.40,RO,Pensiunea Sophia
4,fez,75.20,5.82,0,34.04,-5.00,MA,HÔTEL SAHRAI
5,bejar,73.99,3.67,0,40.39,-5.76,ES,Hotel Cinco Castaños
6,novo aripuana,71.89,0.16,0,-5.12,-60.38,BR,Hotel Gonçalves
7,oriximina,74.97,2.35,0,-1.77,-55.87,BR,Hotel Mapuera
8,jacareacanga,70.03,1.48,0,-6.22,-57.75,BR,HOTEL DA TANIA
9,bolu,76.69,9.86,0,40.67,31.58,TR,Gazelle Resort&SPA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [17]:
hotel_names = hotel_df["Hotel Name"].tolist()

In [18]:
# Add marker layer on top of heat map

fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_names])
fig.add_layer(markers)

# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))